# Import packages

In [ ]:
# detect if running in colab
try:
    import google.colab

    ! pip install torchmetrics
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

import gc
import math
import os
import random
import sys
import time
from pprint import pprint

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import wandb
from datasets import load_from_disk
from PIL.TiffImagePlugin import TiffImageFile
from torch.utils.data import DataLoader, Dataset
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torchvision import tv_tensors
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import v2 as T
from tqdm.notebook import tqdm

# --- A100 OPTIMIZATION: ENABLE TF32 ---
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount("/content/drive")
    BASE_PATH = "/content/drive/MyDrive/datasets/SelvaBox/saved/"
else:
    BASE_PATH = "../data/selvabox/"

In [ ]:
def setup_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True


setup_seed(42)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data

In [ ]:
hf_train_ds = load_from_disk(BASE_PATH + "train")
hf_val_ds = load_from_disk(BASE_PATH + "validation")
hf_test_ds = load_from_disk(BASE_PATH + "test")

In [ ]:
print(f"Number of training samples: {len(hf_train_ds)}")
print(f"Number of validation samples: {len(hf_val_ds)}")
print(f"Number of test samples: {len(hf_test_ds)}")

# Utility functions

In [ ]:
class EarlyStopper:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = -float(
            "inf"
        )  # Looking for max mAP, so init with negative inf
        self.early_stop = False

    def __call__(self, current_score):
        # Logic for maximizing metric (mAP)
        if current_score > (self.best_score + self.min_delta):
            self.best_score = current_score
            self.counter = 0
        else:
            self.counter += 1
            print(
                f"   --> EarlyStopping counter: {self.counter} out of {self.patience}"
            )
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
def plot_image(
    img, boxes, scores=None, labels=None, class_names=None, save_path=None, show=True
):
    """
    Plots bounding boxes on an image with optional scores and labels.

    Args:
        img (np.array | torch.Tensor): Input image. Shape [H, W, C] (numpy) or [C, H, W] (torch).
        boxes (np.array | torch.Tensor): Bounding boxes [N, 4] format (xmin, ymin, xmax, ymax).
        scores (np.array | torch.Tensor, optional): Confidence scores [N]. Defaults to None.
        labels (np.array | torch.Tensor, optional): Class indices [N]. Defaults to None.
        class_names (list, optional): List of class string names. Defaults to None.
        save_path (str, optional): Path to save the figure. Defaults to None.
        show (bool, optional): Whether to display the plot. Defaults to True.
    """

    # --- 1. Data Standardization ---
    # Convert PyTorch tensors to Numpy if necessary
    if isinstance(img, torch.Tensor):
        img = img.cpu().numpy()
        # If image is [C, H, W], transpose to [H, W, C] for Matplotlib
        if img.shape[0] < img.shape[2]:
            img = img.transpose(1, 2, 0)

    if isinstance(boxes, torch.Tensor):
        boxes = boxes.cpu().numpy()

    if isinstance(scores, torch.Tensor):
        scores = scores.cpu().numpy()

    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()

    # Normalize image range if it's float 0-1, mostly for display consistency
    # (Matplotlib handles 0-1 floats or 0-255 ints, but mixing is bad)
    if img.dtype == np.float32 or img.dtype == np.float64:
        img = np.clip(img, 0, 1)

    # --- 2. Setup Figure ---
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(img)

    # --- 3. Color Setup ---
    # If no class names provided, default to a generic list
    if class_names is None:
        if labels is not None:
            max_label = int(np.max(labels))
            class_names = [f"Class {i}" for i in range(max_label + 1)]
        else:
            class_names = ["Object"]

    # Generate distinct colors for classes
    cmap = plt.get_cmap("tab20b")
    colors = [cmap(i) for i in np.linspace(0, 1, len(class_names))]

    # --- 4. Plotting Loop ---
    for i, box in enumerate(boxes):
        xmin, ymin, xmax, ymax = box

        # Determine Label
        if labels is not None:
            cls_id = int(labels[i])
        else:
            cls_id = 0  # Default to 0 if no labels provided

        color = colors[cls_id % len(colors)]
        class_name = (
            class_names[cls_id] if cls_id < len(class_names) else f"Class {cls_id}"
        )

        # Draw Rectangle
        width = xmax - xmin
        height = ymax - ymin
        rect = patches.Rectangle(
            (xmin, ymin), width, height, linewidth=2, edgecolor=color, facecolor="none"
        )
        ax.add_patch(rect)

        # Build Text String
        display_text = class_name
        if scores is not None:
            display_text += f" {int(100 * scores[i])}%"

        # Draw Text with background
        ax.text(
            xmin,
            ymin,
            display_text,
            color="white",
            fontsize=10,
            verticalalignment="top",
            bbox={
                "color": color,
                "pad": 2,
                "alpha": 0.8,
            },  # Added alpha for better visibility
        )

    plt.axis("off")  # Hide axes ticks

    if save_path:
        plt.savefig(save_path, bbox_inches="tight")

    if show:
        plt.show()
    else:
        plt.close()

# Hyper-parameters

In [ ]:
CONFIG = {
    "project_name": "selva-box-tree-detection",  # WandB project name
    "name": "fasterrcnn-min-size-max-size-experiment",
    "num_classes": 2,  # Background + your classes (e.g., 1 class + 1 background = 2)
    "batch_size": 16,
    "num_workers": 4,
    "num_epochs": 20,
    "learning_rate": 0.005,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "step_size": 3,  # Scheduler step size
    "gamma": 0.1,  # Scheduler gamma
    "patience": 5,  # Early stopping patience
    "device": device,
    "model_name": "fasterrcnn_resnet50_fpn",
    "model_min_size": 1024,
    "model_max_size": 2000,
}

In [ ]:
wandb.init(
    project=CONFIG["project_name"],
    name=CONFIG["name"],
    config=CONFIG,
)

# Custom dataset

In [ ]:
# inspired from: https://docs.pytorch.org/tutorials/intermediate/torchvision_tutorial.html
class SelvaBoxDataset(Dataset):
    def __init__(self, hf_dataset, n_classes=1, transforms=None):
        self.dataset = hf_dataset
        self.n_classes = n_classes
        self.transforms = transforms

    def __getitem__(self, index):
        sample = self.dataset[index]
        image: TiffImageFile = sample["image"]
        annotations_dict = sample["annotations"]

        if image.mode != "RGB":
            image = image.convert("RGB")

        # PIL returns (Width, Height)
        w, h = image.size

        image = tv_tensors.Image(image)

        # number of objects/trees in the image
        num_objs = len(annotations_dict["bbox"])

        target = {
            "boxes": tv_tensors.BoundingBoxes(
                data=annotations_dict["bbox"],
                format="XYWH",  # COCO format
                canvas_size=(h, w),
            ),
            "labels": torch.ones((num_objs,), dtype=torch.int64),
            "image_id": torch.tensor(
                index
            ),  # TODO: is this necessary? when moving data to GPU, it expects a tensor
            "area": torch.tensor(annotations_dict["area"], dtype=torch.float32),
            "iscrowd": torch.tensor(annotations_dict["iscrowd"], dtype=torch.int64),
        }

        if self.transforms:
            image, target = self.transforms(image, target)

        if target["boxes"].shape[0] == 0:
            # Ensure it retains the correct shape for the collate_fn and model
            target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
            target["labels"] = torch.zeros((0,), dtype=torch.int64)
            target["area"] = torch.zeros((0,), dtype=torch.float32)
            target["iscrowd"] = torch.zeros((0,), dtype=torch.int64)

        return image, target

    def __len__(self):
        return len(self.dataset)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
# For training
train_transforms = T.Compose(
    [
        T.RandomHorizontalFlip(0.5),
        T.RandomVerticalFlip(0.5),
        T.ConvertBoundingBoxFormat(format="XYXY"),
        T.ToDtype(torch.float, scale=True),
        T.ToPureTensor(),
    ]
)

# For validation/test
eval_transforms = T.Compose(
    [
        T.ConvertBoundingBoxFormat(format="XYXY"),
        T.ToDtype(torch.float, scale=True),
        T.ToPureTensor(),
    ]
)

In [ ]:
train_dataset = SelvaBoxDataset(hf_train_ds, transforms=train_transforms)
val_dataset = SelvaBoxDataset(hf_val_ds, transforms=eval_transforms)
test_dataset = SelvaBoxDataset(hf_test_ds, transforms=eval_transforms)

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG["batch_size"],
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=CONFIG["num_workers"],
    pin_memory=True,
    persistent_workers=True,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG["batch_size"],
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=CONFIG["num_workers"],
    pin_memory=True,
    persistent_workers=True,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG["batch_size"],
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=CONFIG["num_workers"],
    pin_memory=True,
    persistent_workers=True,
)

# Model definition

In [ ]:
def get_model(num_classes, min_size=1024, max_size=2000):
    # Load a model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
        weights="DEFAULT", min_size=min_size, max_size=max_size
    )
    
    # Replace the classifier with a new one, that has num_classes
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

# Model training

In [ ]:
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()

    running_loss = 0.0
    running_cls_loss = 0.0
    running_box_reg_loss = 0.0

    for i, (images, targets) in tqdm(
        enumerate(data_loader), total=len(data_loader), desc=f"Training Epoch {epoch}"
    ):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()

        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        loss_value = losses.item()

        # Specific losses for detailed logging
        cls_loss = loss_dict["loss_classifier"].item()
        box_reg_loss = loss_dict["loss_box_reg"].item()

        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(
                f"Loss Dict: {loss_dict}"
            )  # Added print to see which specific loss failed
            sys.exit(1)

        losses.backward()

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)

        optimizer.step()

        running_loss += loss_value
        running_cls_loss += cls_loss
        running_box_reg_loss += box_reg_loss

        if i % 10 == 0:
            wandb.log(
                {
                    "train/batch_loss": loss_value,
                    "train/batch_cls_loss": cls_loss,
                    "train/batch_box_loss": box_reg_loss,
                }
            )

    epoch_loss = running_loss / len(data_loader)
    return epoch_loss


@torch.no_grad()
def evaluate_map(model, data_loader, device):
    model.eval()
    
    metric = MeanAveragePrecision(
        box_format="xyxy",
        iou_type="bbox",
        max_detection_thresholds=[1, 100, 400],
        class_metrics=True,
    )

    for images, targets in tqdm(data_loader, desc="Validating"):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        predictions = model(images)

        predictions = [{k: v.cpu() for k, v in p.items()} for p in predictions]
        targets_cpu = [{k: v.cpu() for k, v in t.items()} for t in targets]

        metric.update(predictions, targets_cpu)

    results = metric.compute()
    return results

In [ ]:
model = get_model(
    CONFIG["num_classes"], min_size=CONFIG["min_size"], max_size=CONFIG["max_size"]
)
model.to(device)

In [ ]:
# 4. Optimizer & Scheduler
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.SGD(
    params,
    lr=CONFIG["learning_rate"],
    momentum=CONFIG["momentum"],
    weight_decay=CONFIG["weight_decay"],
)

lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=CONFIG["step_size"], gamma=CONFIG["gamma"]
)

early_stopper = EarlyStopper(patience=CONFIG["patience"])

In [ ]:
print("Starting training...")

best_map = 0.0
max_peak_mem = 0.0

# Setup checkpoint directory
if IN_COLAB:
    checkpoint_dir = "/content/drive/MyDrive/models/selvabox/"
    os.makedirs(checkpoint_dir, exist_ok=True)
else:
    checkpoint_dir = "../models/"
    os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, "faster_r_cnn_larger_input_size.pth")
print(f"Checkpoint will be saved to: {checkpoint_path}")

# Memory tracking
torch.cuda.reset_peak_memory_stats()
start_train_time = time.time()

for epoch in tqdm(range(CONFIG["num_epochs"]), desc="Overall Training Progress"):
    epoch_start = time.time()

    # --- Train ---
    avg_train_loss = train_one_epoch(model, optimizer, train_loader, device, epoch)

    # --- Validation (Loss Proxy) ---
    val_metrics = evaluate_map(model, val_loader, device)
    val_map_50 = val_metrics["map_50"].item()
    val_map = val_metrics["map"].item()

    # --- Update Learning Rate ---
    lr_scheduler.step()
    curr_lr = optimizer.param_groups[0]["lr"]

    epoch_end = time.time()
    epoch_duration = (epoch_end - epoch_start) / 60
    peak_mem = torch.cuda.max_memory_allocated() / 1024 / 1024
    max_peak_mem = max(max_peak_mem, peak_mem)

    print(
        f"Epoch [{epoch + 1}/{CONFIG['num_epochs']}] "
        f"Train Loss: {avg_train_loss:.4f} | "
        f"Val mAP: {val_map:.6f} | "
        f"Val mAP_50: {val_map_50:.4f} | "
        f"Time: {epoch_duration:.1f}m | "
        f"Peak Mem: {peak_mem:.0f} MB"
    )

    # --- Logging ---
    wandb.log(
        {
            "epoch": epoch + 1,
            "train/epoch_loss": avg_train_loss,
            "val/mAP": val_map,
            "val/mAP_50": val_map_50,
            "learning_rate": curr_lr,
            "system/peak_mem_mb": peak_mem,
            "system/max_peak_mem_mb": max_peak_mem,
        }
    )

    # --- Save Best Model ---
    if val_map > best_map:
        best_map = val_map
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "map": best_map,
                "config": CONFIG,
            },
            checkpoint_path,
        )
        print(f"--> New Best Model Saved (mAP: {best_map:.4f})")

    # 6. Early Stopping
    early_stopper(val_map)
    if early_stopper.early_stop:
        print("--> Early stopping triggered.")
        break

    # 7. Flush Memory
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()


total_train_time = (time.time() - start_train_time) / 60

print("Training complete.")
print(f"Total training time: {total_train_time:.1f} minutes")
print(f"Best validation mAP: {best_map:.4f}")
print(f"Memory Peak: {peak_mem:.0f} MB")

In [ ]:
wandb.log(
    {
        "training/total_time_minutes": total_train_time,
        "training/best_val_map": best_map,
        "system/overall_peak_mem_mb": max_peak_mem,
    }
)

# Model evaluation

In [ ]:
model = get_model(
    CONFIG["num_classes"],
    min_size=CONFIG["model_min_size"],
    max_size=CONFIG["model_max_size"],
)

In [ ]:
# Load best model from checkpoint
if IN_COLAB:
    checkpoint_path = (
        "/content/drive/MyDrive/models/selvabox/faster_r_cnn_larger_input_size.pth"
    )
else:
    checkpoint_path = "../models/faster_r_cnn_larger_input_size.pth"

print(f"Loading checkpoint from: {checkpoint_path}")
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint["model_state_dict"])
model.to(device)

print(
    f"Loaded model from epoch {checkpoint['epoch']} with mAP: {checkpoint['map']:.4f}"
)


In [ ]:
# Initialize the metric
metric = MeanAveragePrecision(
    box_format="xyxy",
    iou_type="bbox",
    max_detection_thresholds=[1, 100, 400],
    class_metrics=True,
)

model.eval()

# Assuming test_loader is defined
for images, targets in tqdm(test_loader, desc="Testing"):
    images = list(image.to(device) for image in images)

    # Forward pass
    with torch.no_grad():
        predictions = model(images)

    # Move to CPU (torchmetrics handles CPU/GPU, but consistency is good)
    predictions = [{k: v.cpu() for k, v in p.items()} for p in predictions]

    # Update the metric with this batch
    # targets need to be a list of dicts on the same device as predictions
    # If targets are on GPU, move to CPU to match predictions
    targets_cpu = [{k: v.cpu() for k, v in t.items()} for t in targets]

    metric.update(predictions, targets_cpu)

    # Clear GPU cache to prevent OOM errors
    torch.cuda.empty_cache()

# Compute the final metrics over the whole dataset
results = metric.compute()

# Print results
print(f"mAP (IoU=0.50:0.95): {results['map']:.4f}")
print(f"mAP (IoU=0.50): {results['map_50']:.4f}")
print(f"mAP (IoU=0.75): {results['map_75']:.4f}")

pprint(results)

In [ ]:
wandb.log(results)

In [ ]:
wandb.finish()

# Visualize results

In [ ]:
# visualize prediction and ground truth on some test images side by side
model.eval()

test_batch = next(iter(test_loader))  # skip first batch to avoid reusing cached images

images, targets = test_batch

images = list(img.to(device) for img in images)

with torch.no_grad():
    predictions = model(images)

predictions = [{k: v.cpu() for k, v in p.items()} for p in predictions]

images = [img.cpu() for img in images]
targets = [{k: v.cpu() for k, v in t.items()} for t in targets]

for i in range(len(images)):
    img = images[i]
    pred = predictions[i]
    target = targets[i]

    # keep only predictions with score > 0.5
    keep_idxs = pred["scores"] > 0.5
    pred["boxes"] = pred["boxes"][keep_idxs]
    pred["scores"] = pred["scores"][keep_idxs]
    pred["labels"] = pred["labels"][keep_idxs]

    print(f"Image {i + 1} Predictions:")
    plot_image(
        img,
        boxes=pred["boxes"],
        scores=pred["scores"],
        labels=pred["labels"],
        class_names=["tree"],
        show=True,
    )

    print(f"Image {i + 1} Ground Truth:")
    plot_image(
        img,
        boxes=target["boxes"],
        labels=target["labels"],
        class_names=["tree"],
        show=True,
    )